In [1]:
import numpy as np
import tensorflow as tf
import os
import copy
from math import pi
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
from pandas import DataFrame as DF
from models import AP_ResNet, ResNet, VGG_branch, ResNetLSTM

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from classifier import preprocessing as prep
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

2022-08-04 08:00:23.867965: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
class_names = ['asphalt','bicycle','block','floor','ground']
bound = 414

In [3]:
def heatmap(matrix,title):
    df=DF(matrix, index = class_names, columns = class_names)
    plt.figure(figsize=(10,10))
    sns.heatmap(df, annot=True)
    plt.tick_params(axis='x', top=True, labeltop = True,bottom=False, labelbottom=False)
    plt.xticks(np.arange(0.5, len(df.columns), 1), df.columns)
    plt.yticks(np.arange(0.5, len(df.index), 1), df.index)
    plt.xlabel("Prediction",position = (0.5,1.0+0.05))
    plt.ylabel("Object")
    plt.title(title)

In [4]:
label2idx_Dict = {
                'asphalt' : 0,
                'bicycle' : 1,
                'block' : 2,
                'floor' : 3,
                'ground' : 4,
            }

idx2label_Dict = {
    0 : 'asphalt',
    1 : 'bicycle',
    2 : 'block',
    3 : 'floor',
    4 : 'ground',
}

In [5]:
dir_path = './road_data'
def readNpy(dir_path):
    class_num = len(idx2label_Dict)

    Asphalt = list()
    Bicycle = list()
    Block = list()
    Floor = list()
    Ground = list()
    Asphalt = np.array(Asphalt)
    Bicycle = np.array(Bicycle)
    Block = np.array(Block)
    Floor = np.array(Floor)
    Ground = np.array(Ground)
    

    for dir in os.listdir(dir_path):
        d_path = os.path.join(dir_path, dir)
        file_list = os.listdir(d_path)
        for file in file_list:
            file_path = os.path.join(d_path, file)
            if dir == idx2label_Dict[0] :
                if len(Asphalt) == 0:
                    Asphalt = np.load(file_path, allow_pickle=True)
                else :
                    Asphalt = np.append(Asphalt, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[1]:
                if len(Bicycle) == 0:
                    Bicycle = np.load(file_path, allow_pickle=True)
                else :
                    Bicycle = np.append(Bicycle, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[2]:
                if len(Block) == 0:
                    Block = np.load(file_path, allow_pickle=True)
                else:
                    Block = np.append(Block, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[3]:
                if len(Floor) == 0:
                    Floor = np.load(file_path, allow_pickle=True)
                else:
                    Floor = np.append(Floor, np.load(file_path), axis = 0)
            elif dir == idx2label_Dict[4]:
                if len(Ground) == 0:
                    Ground = np.load(file_path, allow_pickle=True)
                else:
                    Ground = np.append(Ground, np.load(file_path), axis = 0)

    bound = Asphalt.shape[1]

    Ground_label = np.full((Ground.shape[0], class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['ground']])
    Asphalt_label = np.full((Asphalt.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['asphalt']])
    Bicycle_label = np.full((Bicycle.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['bicycle']])
    Block_label = np.full((Block.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['block']])
    Floor_label = np.full((Floor.shape[0],class_num), np.eye(len(label2idx_Dict))[label2idx_Dict['floor']])

    Ground = np.concatenate((Ground, Ground_label), axis=1)
    Asphalt = np.concatenate((Asphalt, Asphalt_label), axis=1)
    Bicycle = np.concatenate((Bicycle, Bicycle_label), axis=1)
    Block = np.concatenate((Block, Block_label), axis=1)
    Floor = np.concatenate((Floor, Floor_label), axis=1)
    
    array = Asphalt
    array = np.append(array, Bicycle, axis = 0)
    array = np.append(array, Block, axis = 0)
    array = np.append(array, Floor, axis = 0)
    array = np.append(array, Ground, axis = 0)
    s = np.arange(array.shape[0])
    np.random.shuffle(s)
    array_s = array[s]

    X = array_s[:,:bound]
    Y = np.real(array_s[:,bound:])
    return copy.deepcopy(X), copy.deepcopy(Y)

X, Y = readNpy(dir_path)

In [6]:
def seperater(arr):
    # global maximum
    pre_data = arr
    amp = np.abs(pre_data)
    # amp = amp / maximum
    phs = np.angle(pre_data)
    # phs = (phs - (- pi)) / (pi - (- pi))
    sin = np.sin(phs)
    sin = (sin + 1) / 2
    seperated_data = np.stack((amp.T,sin.T), axis=0)
    seperated_data = np.expand_dims(seperated_data, axis=0)
    return np.array(seperated_data)

In [7]:
def dataSeperator(arr):
    temp = copy.deepcopy(seperater(arr[0]))
    for i in range(1, len(arr)):
        temp = np.concatenate((temp, seperater(arr[i])), axis=0)
    return temp

In [8]:
Split_X = dataSeperator(X)

In [9]:
checkpoint_filepath = 'road_detection_lstm'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model/' + checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only = True,
    save_weigths_only = False,
)
log_dir = './logs/fit/'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [10]:
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data[0])-seq_length-1):
        x = data[0][:][i:(i+seq_length)]
        y = data[1][i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [11]:
seq_length = 10
X, Y = create_sequences((Split_X, Y), seq_length = seq_length)

In [12]:
print(X.shape, Y.shape)

(2389, 10, 2, 414) (2389, 5)


In [13]:
def train(model, X, Y, history_dict=None):
    Epoch = 100
    callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = './model/' + 'Resnet_LSTM',
        monitor='val_accuracy',
        mode='max',
        save_best_only = True,
        save_weigths_only = False,
    )
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.CategoricalCrossentropy()
    model.compile(optimizer = optimizer , loss = loss, 
                metrics = ['accuracy', 'categorical_crossentropy'])
    history = model.fit(X, Y,  batch_size = 64, epochs = Epoch,
                callbacks = callback, validation_split = 0.3)
    return history

In [14]:
model = ResNetLSTM()
history = train(model, X, Y)

2022-08-04 08:00:30.099096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-04 08:00:30.617355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22304 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d8:00.0, compute capability: 8.6


Epoch 1/100


2022-08-04 08:00:34.737010: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8301
2022-08-04 08:00:36.406935: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


27/27 [==============================] - ETA: 0s - loss: 3.0238 - accuracy: 0.1866 - categorical_crossentropy: 3.0238

INFO:tensorflow:Assets written to: ./model/Resnet_LSTM/assets


...

INFO:tensorflow:Assets written to: ./model/Resnet_LSTM/assets


27/27 [==============================] - 10s 389ms/step - loss: 1.8389 - accuracy: 0.2458 - categorical_crossentropy: 1.8389 - val_loss: 1.5975 - val_accuracy: 0.2469 - val_categorical_crossentropy: 1.5975
Epoch 12/100
27/27 [==============================] - 3s 99ms/step - loss: 1.8550 - accuracy: 0.2177 - categorical_crossentropy: 1.8550 - val_loss: 1.5938 - val_accuracy: 0.2455 - val_categorical_crossentropy: 1.5938
Epoch 13/100
27/27 [==============================] - 3s 94ms/step - loss: 1.8003 - accuracy: 0.2380 - categorical_crossentropy: 1.8003 - val_loss: 1.5940 - val_accuracy: 0.2385 - val_categorical_crossentropy: 1.5940
Epoch 14/100
27/27 [==============================] - 3s 96ms/step - loss: 1.7446 - accuracy: 0.2255 - categorical_crossentropy: 1.7446 - val_loss: 1.5910 - val_accuracy: 0.2385 - val_categorical_crossentropy: 1.5910
Epoch 15/100
27/27 [==============================] - 3s 97ms/step - loss: 1.7131 - accuracy: 0.2416 - categorical_crossentropy: 1.7131 - val_l

INFO:tensorflow:Assets written to: ./model/Resnet_LSTM/assets


INFO:tensorflow:Assets written to: ./model/Resnet_LSTM/assets


27/27 [==============================] - 11s 402ms/step - loss: 1.5956 - accuracy: 0.2560 - categorical_crossentropy: 1.5956 - val_loss: 1.5824 - val_accuracy: 0.2692 - val_categorical_crossentropy: 1.5824
Epoch 28/100
27/27 [==============================] - 3s 95ms/step - loss: 1.5928 - accuracy: 0.2309 - categorical_crossentropy: 1.5928 - val_loss: 1.5816 - val_accuracy: 0.2245 - val_categorical_crossentropy: 1.5816
Epoch 29/100
27/27 [==============================] - 3s 99ms/step - loss: 1.5744 - accuracy: 0.2566 - categorical_crossentropy: 1.5744 - val_loss: 1.5808 - val_accuracy: 0.2497 - val_categorical_crossentropy: 1.5808
Epoch 30/100
27/27 [==============================] - 3s 99ms/step - loss: 1.5779 - accuracy: 0.2500 - categorical_crossentropy: 1.5779 - val_loss: 1.5786 - val_accuracy: 0.2469 - val_categorical_crossentropy: 1.5786
Epoch 31/100
27/27 [==============================] - 3s 94ms/step - loss: 1.5858 - accuracy: 0.2614 - categorical_crossentropy: 1.5858 - val_l

In [15]:
model.summary()

Model: "res_net_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  32        
                                                                 
 resnet_layer2d (ResnetLayer  multiple                 3120      
 2D)                                                             
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 resnet_layer2d_1 (ResnetLay  multiple                 11872     
 er2D)                                                           
                                                                 
 max_pooling2d_1 (MaxPooling  multiple                 0         
 2D)                                                  